# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_data_df= pd.read_csv("../Output/city_data.csv")
city_data_df = city_data_df.dropna()
city_data_df

,Unnamed: 0,City,City Name,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country
1,1,3449747,Santa Vitória do Palmar,-33.5189,-53.3681,57.38,78,79,9.53,BR
2,2,2194098,Ahipara,-35.1667,173.1667,52.23,83,81,9.48,NZ
3,3,3369157,Cape Town,-33.9258,18.4232,53.47,66,0,5.75,ZA
4,4,6457405,Province of Vibo Valentia,38.6996,16.1274,71.80,72,24,10.51,IT
5,5,3652764,Puerto Ayora,-0.7393,-90.3518,74.55,85,82,5.01,EC
...,...,...,...,...,...,...,...,...,...,...
545,545,1513131,Navoiy,40.0844,65.3792,78.28,18,100,18.41,UZ
546,546,1513491,Yangi Marg`ilon,40.4272,71.7189,77.72,44,85,4.61,UZ
547,547,7872050,Enns,48.2141,14.4778,67.33,87,86,4.88,AT
548,548,217745,Bumba,2.1833,22.4667,80.85,53,69,3.67,CD


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
#config gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
location = city_data_df[["Lat","Lng"]].astype(float)
humidity =city_data_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
new_city_df = city_data_df.loc[(city_data_df["Wind Speed"] <= 10) & (city_data_df["Clouds"] == 0) & \
                                   (city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80)]

new_city_df=new_city_df.dropna()
new_city_df                        

,Unnamed: 0,City,City Name,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country
32,32,3466980,Caravelas,-17.7125,-39.2481,78.64,55,0,7.56,BR
38,38,695104,Ruska Poliana,49.4170,31.9201,76.17,53,0,3.13,UA
159,159,304183,Bayir,37.2687,28.2167,77.52,36,0,3.29,TR
176,176,146214,Paphos,34.7667,32.4167,79.88,73,0,4.61,CY
257,257,3045991,Rajka,47.9964,17.1982,78.78,48,0,4.61,HU
515,515,2034754,Shunyi,40.1250,116.6453,78.82,83,0,4.47,CN
523,523,705442,Kulevcha,46.0303,29.9361,76.91,44,0,5.39,UA
537,537,686090,Asău,46.4333,26.4000,71.96,71,0,0.56,RO


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = new_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"]=''
hotel_df

,City,Country,Lat,Lng,Hotel Name
32,3466980,BR,-17.7125,-39.2481,
38,695104,UA,49.4170,31.9201,
159,304183,TR,37.2687,28.2167,
176,146214,CY,34.7667,32.4167,
257,3045991,HU,47.9964,17.1982,
515,2034754,CN,40.1250,116.6453,
523,705442,UA,46.0303,29.9361,
537,686090,RO,46.4333,26.4000,


In [25]:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


hotel_name = requests.get(base_url, params=params)

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        #time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 32: 3466980.
Closest hotel in 3466980 is Pousada Liberdade.
--------------------Ending Search
Retrieving Results for Index 38: 695104.
Closest hotel in 695104 is Vershnyk.
--------------------Ending Search
Retrieving Results for Index 159: 304183.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 176: 146214.
Closest hotel in 146214 is Elysium Hotel.
--------------------Ending Search
Retrieving Results for Index 257: 3045991.
Closest hotel in 3045991 is Ponteo Activity Park.
--------------------Ending Search
Retrieving Results for Index 515: 2034754.
Closest hotel in 2034754 is Lailai Hotel.
--------------------Ending Search
Retrieving Results for Index 523: 705442.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 537: 686090.
Closest hotel in 686090 is Pensiunea Sophia.
--------------------Ending Search


In [27]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
32,3466980,BR,-17.7125,-39.2481,Pousada Liberdade
38,695104,UA,49.4170,31.9201,Vershnyk
159,304183,TR,37.2687,28.2167,
176,146214,CY,34.7667,32.4167,Elysium Hotel
257,3045991,HU,47.9964,17.1982,Ponteo Activity Park
515,2034754,CN,40.1250,116.6453,Lailai Hotel
523,705442,UA,46.0303,29.9361,
537,686090,RO,46.4333,26.4000,Pensiunea Sophia


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))